## Learning FourSquare API with Python

## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.

In [1]:
#Import necessary Libraries

import requests #library to hadle requests
import pandas as pd #for data analysis
import numpy as np #to handle data in a vectorized manner
import random #for random number generation

!pip install geopy
from geopy.geocoders import Nominatim #module to convert an address into latitude and logitude values

#libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

#transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium #plotting library

print('Folium installed')
print('Libraries imported')

     |████████████████████████████████| 92kB 4.6MB/s eta 0:00:01
Folium installed
Libraries imported


In [62]:
#Define Foursquare credentials and version

CLIENT_ID = 'XT0YNJESUHUMDOGSL5P1MMLELXBVIVMULYDPTPXKRYS45SL2'
CLIENT_SECRET = 'LOSOF50ZNJFHFPN5K1IQ0TYS412YOFIZCOOHJMTIQHASMIZY'
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT ID:' + CLIENT_ID)
print('CLEINT SECRET' + CLIENT_SECRET)

Your credentials:
CLIENT ID:XT0YNJESUHUMDOGSL5P1MMLELXBVIVMULYDPTPXKRYS45SL2
CLEINT SECRETLOSOF50ZNJFHFPN5K1IQ0TYS412YOFIZCOOHJMTIQHASMIZY


**Assume that you're staying at the Conrad hotel. Convert the address to its latitude and longitude coordinates**

In [63]:
#in order to define an instance of the geocorder, you need to define a user_agent

address = '102 North End Ave, New York, NY'

geolocater = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


**Define a query to search for Italian food that is within 500 meters from the Conrad Hotel**

In [64]:
search_query = 'Italian'
radius = 500
print(search_query + '.... OK!')

Italian.... OK!


In [65]:
#Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XT0YNJESUHUMDOGSL5P1MMLELXBVIVMULYDPTPXKRYS45SL2&client_secret=LOSOF50ZNJFHFPN5K1IQ0TYS412YOFIZCOOHJMTIQHASMIZY&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

In [66]:
#Send the GET Request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea72be1f7706a7e2bc691e2'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '29454

In [67]:
#Get relevant part of JSON and transform it into a pandas dataframe

#1.assign relevant part of JSON to venues
venues = results['response']['venues']

#2.transform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}]",294544,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-pizza-bar-225-murray-st-new-york/294544?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294544,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United States]","[{'label': 'display', 'lat': 40.71521779064671, 'lng': -74.01473940209351}, {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}]",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1588014029
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'Food', 'pluralName': 'Food', 'shortName': 'Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,469,"[63 W Broadway, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.714484000000006, 'lng': -74.00980600000001}]",40.714484,-74.009806,10007,NY,Conca Cucina Italian Restaurant,v-1588014029
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}]",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,"[124 Chambers St (btwn Church St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71533713859952, 'lng': -74.00884766217825}, {'label': 'entrance', 'lat': 40.715202, 'lng': -74.008779}]",40.715337,-74.008848,10007,NY,Ecco,v-1588014029


In [68]:
#Define information of interest and filter dataframe

#1.keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

#2.function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#3.filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

#4.clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United States]","[{'label': 'display', 'lat': 40.71521779064671, 'lng': -74.01473940209351}, {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}]",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,469,"[63 W Broadway, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.714484000000006, 'lng': -74.00980600000001}]",40.714484,-74.009806,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,"[124 Chambers St (btwn Church St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71533713859952, 'lng': -74.00884766217825}, {'label': 'entrance', 'lat': 40.715202, 'lng': -74.008779}]",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


In [69]:
#visualize the Italian restautants that are nearby

dataframe_filtered.name
print(dataframe_filtered.name)

#1.generate map centered around the Conrad Hotel
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

#2.add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

#3.add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    color='blue',
    popup=label,
    fill=True,
    fill_color='blue',
    fill_opacity=0.6
).add_to(venues_map)
    
#4.displat map
venues_map

0    Harry's Italian Pizza Bar      
1    Conca Cucina Italian Restaurant
2    Ecco                           
Name: name, dtype: object


## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Explore the closest Italian restaurant - "Harry's Italian Pizza Bar"

In [70]:
venue_id = '4fa862b3e4b0ebff2f749f06' #ID of Harry's Italian Pizza Bar

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=XT0YNJESUHUMDOGSL5P1MMLELXBVIVMULYDPTPXKRYS45SL2&client_secret=LOSOF50ZNJFHFPN5K1IQ0TYS412YOFIZCOOHJMTIQHASMIZY&v=20180604'

In [71]:
# Send GET request for result
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': '?', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': '

### B. Get the venue's overall rating

In [72]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet')

6.4


This is not a very good rating. Check the rating of the second closest Italian restaurant.

In [73]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, check the third restaurant.

In [74]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


Since this restaurant has a slightly better rating, explore it further.

### C. Get the number of tips

In [75]:
result['response']['venue']['tips']['count']

19

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

In [76]:
#Create URL and send GET request. Make sure to set limit to get all tips

##Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea72b4cb9a3890028aeeaf8'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'E',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [77]:
#Get tips and list of associated features

tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [78]:
#Format column width and display all tips
pd.set_option('display.max_colwidth', -1)

#normalize
tips_df = json_normalize(tips)

#columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,E,484542633


Remember that because we are using a personal developer account, we can access only 2 of the restaurant's tips, instead of all 15 tips.

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

In [79]:
#Define URL, send GET request and display features associated with user

user_id = '484542633' # user ID with most agree counts and complete profile
ACCESS_TOKEN = 'RNNHJB1BJWV5HZHUGKUZLKUNYGS2FYPONGEXAHDEZTMDSPKA' #see https://developer.foursquare.com/docs/places-api/authentication/

# define URL
url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION)

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'checkinPings', 'pings', 'type', 'mayorships', 'checkins', 'lists', 'blockedStatus', 'lenses'])

In [80]:
#User information

print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: Nick
Last Name: El-Tawil
Home City: New York, NY


In [81]:
#Check how many tips this user has submitted
user_data['tips']

{'count': 98}

In [82]:
#Get User's tips

#1.define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

#2.send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

#3.format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

#4.filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

#5.display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
1,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",2,0,5acbec70a0215b732e264fe8
2,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
3,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
4,Burger game strong 💪,1,0,5ab575fb6bdee65f759da8c1
5,"Great burgers & fries! Also, this place is exactly what it’s like when you go to a bar in the Southwest. Source: I’m from Arizona.",2,0,5ab5575d73fe2516ad8f363b
6,Açaí bowl + peanut butter + whey protein = 💪💪💪,1,0,5ab42db53c858d64af2688a4
7,Highly underrated and way less crowded than Central Park!,3,0,5ab42c396f706a29f53ad1a8
8,"Way easier to navigate than the Met proper, plus the Met Breuer focuses on modern art. If I only have a limited amount of time to spend in a museum, I would rather go here than anywhere else!",6,0,5ab42b987dc9e17930e5ff5b
9,Get the açaí bowl with peanut butter after your work out and thank me later 👌,1,0,5ab42aca2a7ab6333652b266


In [83]:
#Get the venue for the tip with the greatest number of agree acounts

tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


In [84]:
#Get user's friends
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

In [85]:
#Retrieve the User's Profile Image
user_data

{'id': '484542633',
 'firstName': 'Nick',
 'lastName': 'El-Tawil',
 'gender': 'male',
 'canonicalUrl': 'https://foursquare.com/user/484542633',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg'},
 'friends': {'count': 0,
  'groups': [{'type': 'friends',
    'name': 'Mutual friends',
    'count': 0,
    'items': []},
   {'type': 'others', 'name': 'Other friends', 'count': 0, 'items': []}]},
 'tips': {'count': 98},
 'homeCity': 'New York, NY',
 'bio': 'https://www.tawil.team/nick-el-tawil/',
 'contact': {'twitter': 'nickeltawil'},
 'photos': {'count': 2,
  'items': [{'id': '5e41a7ce346b7c00085b0f70',
    'createdAt': 1581361102,
    'source': {'name': 'Foursquare for iOS',
     'url': 'https://foursquare.com/download/#/iphone'},
    'prefix': 'https://fastly.4sqi.net/img/general/',
    'suffix': '/484542633_cF12vc8JQJTd-hyznst-CsHHTdSoi4XwCGKhTuQXNfo.jpg',
    'width': 1440,
   

In [86]:
# 1. grab prefix of photo
# 2. set image size
# 3. grab suffix of photo
# 4. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/user/300x300/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg')

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

**Popular spots around the restaurant - "Ecco Restaurant"**

In [87]:
#get the latitude and longitude values of Ecco Restaurant
latitude = 40.715337
longitude = -74.008848

#define URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XT0YNJESUHUMDOGSL5P1MMLELXBVIVMULYDPTPXKRYS45SL2&client_secret=LOSOF50ZNJFHFPN5K1IQ0TYS412YOFIZCOOHJMTIQHASMIZY&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

In [88]:
#send GET request and examine results
import requests

results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [89]:
#get relevant part of JSON
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '54148bc6498ea7bb8c05b70a',
  'name': 'Juice Press',
  'location': {'address': '83 Murray St',
   'crossStreet': 'btwn Greenwich St & W Broadway',
   'lat': 40.71478769908051,
   'lng': -74.0111317502157,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71478769908051,
     'lng': -74.0111317502157}],
   'distance': 202,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['83 Murray St (btwn Greenwich St & W Broadway)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
  

In [90]:
#Process JSON and convert it to a clean dataframe

#1.flatten JSON
dataframe = json_normalize(items)

#2.filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

#3.filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

#4.clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
1,Korin,Furniture / Home Store,57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': '?', 'lat': 40.714727, 'lng': -74.009399}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
2,Los Tacos No. 1,Taco Place,136 Church St,US,New York,United States,NaN,119,"[136 Church St, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",40.714267,-74.008756,NaN,10007,NY,5d5f24ec09484500079aee00
3,Takahachi Bakery,Bakery,25 Murray St,US,New York,United States,at Church St,187,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': '?', 'lat': 40.713716, 'lng': -74.008443}]",40.713653,-74.008804,NaN,10007,NY,4c154c9a77cea593c401d260
4,Takahachi,Sushi Restaurant,145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': '?', 'lat': 40.716508, 'lng': -74.007989}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
5,Exceed Physical Culture,Gym / Fitness Center,97 Reade St,US,New York,United States,bet W Broadway & Church St,79,"[97 Reade St (bet W Broadway & Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.7156286200256, 'lng': -74.0079922583853}, {'label': '?', 'lat': 40.715589, 'lng': -74.008105}]",40.715629,-74.007992,Tribeca,10013,NY,53910ac3498e57a5dc0eb160
6,Equinox Tribeca,Gym,54 Murray St,US,New York,United States,at W Broadway,154,"[54 Murray St (at W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",40.714099,-74.009686,NaN,10007,NY,4a6e331af964a52031d41fe3
7,Whole Foods Market,Grocery Store,270 Greenwich Street,US,New York,United States,at Warren St,214,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715579155420606, 'lng': -74.01136823958119}]",40.715579,-74.011368,Tribeca,10007,NY,49bc3b0af964a52020541fe3
8,Heyday,Spa,92 Reade St,US,New York,United States,NaN,100,"[92 Reade St, New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}, {'label': '?', 'lat': 40.715654, 'lng': -74.00782}]",40.715726,-74.007767,NaN,10013,NY,57ad129c498e05b086594d72
9,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}, {'label': '?', 'lat': 40.715696, 'lng': -74.00988}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3


In [92]:
#visualize above items on the map around our location

venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


#1.add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


#2.add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

#3.display map
venues_map

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

**Now, instead of simply exploring the area around Ecco, explore the venues tha are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. Then get the trending venues around Ecco.**

In [93]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea72c44d03993001b48d162'},
 'response': {'venues': []}}

In [95]:
#Check if any venues are trending at this time
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
    
#display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

In [96]:
#Visualize trending venues
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'